## Deep Ensemble with Imagenet

In [ ]:
import torch
import torchvision
from torch import nn, optim, autograd
from torch.nn import functional as F
from torchvision import transforms, datasets
import numpy as np
from sklearn.metrics import roc_auc_score
import scipy
from utils.LB_utils import * 
import utils.LB_utils_special as LB_utils_special
from utils.load_not_MNIST import notMNIST
import os
import time
import matplotlib.pyplot as plt
import laplace

s = 1
np.random.seed(s)
torch.manual_seed(s)
torch.cuda.manual_seed(s)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
#load imagenet
#imagenet_train_root = os.path.abspath('your imagenet path')
#imagenet_val_root = os.path.abspath('your imagenet path')

# Get the current working directory (directory of your notebook)
notebook_directory = os.path.dirname(os.path.abspath("deep_ensemble_imagenet.ipynb"))

# Navigate to the parent folder (assuming "usr" and "home" are at the same level)
parent_directory = os.path.abspath(os.path.join(notebook_directory, "..","..","..","..","..","..",".."))
# Construct the path to the ImageNet directory
imagenet_directory = os.path.abspath(os.path.join(parent_directory, "/usr/data1/imagenet"))

# print(imagenet_directory)

imagenet_train_root = os.path.abspath(imagenet_directory + '/train')
imagenet_val_root = os.path.abspath(imagenet_directory + '/val')


normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

transform_imagenet_train = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])

transform_imagenet_val =  transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ])

imagenet_train = datasets.ImageFolder(imagenet_train_root, transform=transform_imagenet_train)
indices_small = np.random.choice(np.arange(0, len(imagenet_train)), size=(500,), replace=False)
imagenet_train_small = torch.utils.data.Subset(imagenet_train, torch.tensor(indices_small))
imagenet_val = datasets.ImageFolder(imagenet_val_root, transform=transform_imagenet_val)

train_loader = torch.utils.data.DataLoader(
        imagenet_train_small,
        batch_size=16,
        shuffle=True)

train_loader_16 = torch.utils.data.DataLoader(
        imagenet_train_small,
        batch_size=16,
        shuffle=True)

train_loader_64 = torch.utils.data.DataLoader(
        imagenet_train_small,
        batch_size=64,
        shuffle=True)

train_loader_128 = torch.utils.data.DataLoader(
        imagenet_train_small,
        batch_size=128,
        shuffle=True)

val_loader = torch.utils.data.DataLoader(
        imagenet_val,
        batch_size=64)

In [4]:
import matplotlib
import tueplots
from tueplots import bundles
plt.rcParams.update(tueplots.bundles.icml2022())

print(plt.rcParams['figure.figsize'])
figwidth = plt.rcParams['figure.figsize'][0]
figheight = plt.rcParams['figure.figsize'][1]
matplotlib.rcParams['font.family'] = "serif"
matplotlib.rcParams['font.serif'] = 'Times new Roman'
matplotlib.rcParams['text.usetex'] = True
print(figheight)
print(figwidth)

[3.25, 2.0086104634371584]
2.0086104634371584
3.25


In [5]:
import models
from models.densenet import densenet121
import torch
from torch.utils.data import DataLoader
import push.bayes.ensemble

#load densenet

# densenet = densenet121(pretrained=False).cuda()
# densenet.eval()

epochs = 1

two_particle_params = push.bayes.ensemble.train_deep_ensemble(
        train_loader_128,
        torch.nn.CrossEntropyLoss(),
        epochs,
        densenet121, False,
        num_devices=2,
        num_ensembles=2,
    )

100%|██████████| 1/1 [00:03<00:00,  3.47s/it]
